##1. 머신러닝 기법에 적용하기

In [1]:
import pandas as pd
titanic_url='https://github.com/sehakflower/data/blob/main/titanic_1309.xlsx?raw=true'
titanic=pd.read_excel(titanic_url,sheet_name='total')
train_1000=titanic.iloc[:1000]
test_309=titanic.iloc[1000:]
train_1000.drop(['boat', 'body', 'home.dest'], axis=1, inplace=True)
test_309.drop(['boat', 'body', 'home.dest'], axis=1, inplace=True)
train_df=train_1000
test_df=test_309

# 1. name 열을 5개로 구분하고 숫자형 데이터로 바꾸기
total=[train_df, test_df]
titles={'Mr':1, 'Miss':2, 'Mrs':3, 'Master':4, 'Special':5} # 호칭에 따라 숫자로 표시하기
for dataset in total:
 dataset['title']=dataset.name.str.extract('([A-Za-z]+)\.', expand=False)
 dataset['title']=dataset['title'].replace(['Lady', 'Countess', 'Capt', 'Col',
'Don', 'Dr', 'Major', 'Rev', 'Sir',
'Jonkheer', 'Dona'], 'Special')
 dataset['title']=dataset['title'].replace('Mlle', 'Miss')
 dataset['title']=dataset['title'].replace('Ms', 'Miss')
 dataset['title']=dataset['title'].replace('Mme', 'Mrs')
 dataset['title']=dataset['title'].map(titles) # 타이틀을 숫자로 변경
 dataset['title']=dataset['title'].fillna(0) # NaN을 0으로 변경
train_df=train_df.drop(['name'], axis=1) # 필요 없는 name 열 삭제
test_df=test_df.drop(['name'], axis=1) # 필요 없는 name 열 삭제

# 2.age 열의 빈 값을 호칭별 중앙값으로 바꾸기
train_df['age'].fillna(train_df.groupby('title')['age'].transform('median'), inplace=True)
test_df['age'].fillna(test_df.groupby('title')['age'].transform('median'), inplace=True)
data=[train_df, test_df]
for dataset1 in data:
 dataset1['age']=dataset1['age'].astype(int)
 dataset1.loc[ dataset1['age']<=11, 'age']=0
 dataset1.loc[(dataset1['age']>11)&(dataset1['age']<=18), 'age']=1
 dataset1.loc[(dataset1['age']>18)&(dataset1['age']<=22), 'age']=2
 dataset1.loc[(dataset1['age']>22)&(dataset1['age']<=27), 'age']=3
 dataset1.loc[(dataset1['age']>27)&(dataset1['age']<=33), 'age']=4
 dataset1.loc[(dataset1['age']>33)&(dataset1['age']<=40), 'age']=5
 dataset1.loc[(dataset1['age']>40)&(dataset1['age']<=66), 'age']=6
 dataset1.loc[ dataset1['age']>66, 'age']=6
# 3. sex 열에서 'male':0, 'female':1로 바꾸기
data=[train_df, test_df]
# 일부 전처리된 train_df, test_df를 다시 업데이트해서 더한 후 숫자로 바꾸기 위함
sex_mapping={'male':0, 'female':1}
for dataset in data:
 dataset['sex']=dataset['sex'].map(sex_mapping)
# 4. embarked 열
for dataset in data:
 dataset['embarked']=dataset['embarked'].fillna('S')
embarked_mapping={'S':0, 'C':1, 'Q':2}
for dataset in data:
 dataset['embarked']=dataset['embarked'].map(embarked_mapping)
# 5. sibsp, parch 열
for dataset in data:
 dataset['sibpar']=dataset['sibsp']+dataset['parch']
 dataset.loc[dataset['sibpar']>0, 'n_alone']=0
 dataset.loc[dataset['sibpar']==0, 'n_alone']=1
 dataset['n_alone']=dataset['n_alone'].astype(int)
# 6. fare 열, 개인별 요금 나타내는 fare_person 열 추가하기
train_df["fare"].fillna(train_df.groupby('pclass')['fare'].transform('median'),
inplace=True)
test_df["fare"].fillna(test_df.groupby('pclass')['fare'].transform('median'),
inplace=True)

for dataset in data:
 dataset.loc[ dataset['fare']<=20, 'fare']=1
 dataset.loc[(dataset['fare']>20)&(dataset['fare']<=30), 'fare']=2
 dataset.loc[(dataset['fare']>30)&(dataset['fare']<=50), 'fare']=3
 dataset.loc[(dataset['fare']>50)&(dataset['fare']<=100), 'fare']=4
 dataset.loc[ dataset['fare']>100, 'fare']=5
for dataset1 in data:
 dataset1['fare_person']=dataset1['fare']/(dataset1['sibpar']+1)
 dataset1['fare_person']=dataset1['fare_person'].astype(int)
# 7. 최종 데이터
X_columns=['pclass', 'sex', 'age', 'embarked', 'title', 'sibpar', 'n_alone', 'fare_person']
y_column=['survived']
X_train=train_df[X_columns]
y_train=train_df[y_column]
X_test=test_df[X_columns]
y_test=test_df[y_column]

<ipython-input-1-983165b00f18>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_1000.drop(['boat', 'body', 'home.dest'], axis=1, inplace=True)
<ipython-input-1-983165b00f18>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_309.drop(['boat', 'body', 'home.dest'], axis=1, inplace=True)
<ipython-input-1-983165b00f18>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datas

###필요한 라이브러리

In [2]:
import numpy as np # 선형대수
import pandas as pd # 데이터 처리
# 머신러닝 알고리즘
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
# 시각화
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style

### 의사결정나무

In [3]:
decision_tree=DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
Y_pred=decision_tree.predict(X_test)
train_acc_decision_tree=round(decision_tree.score(X_train, y_train)*100, 2)
test_acc_decision_tree=round(decision_tree.score(X_test, y_test)*100, 2)
train_acc_decision_tree,test_acc_decision_tree

(87.8, 69.58)

### 랜덤 포레스트

In [4]:
random_forest=RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)
Y_prediction=random_forest.predict(X_test)
random_forest.score(X_train, y_train)
train_acc_random_forest=round(random_forest.score(X_train, y_train)*100, 2)
test_acc_random_forest=round(random_forest.score(X_test, y_test)*100, 2)
train_acc_random_forest,test_acc_random_forest

<ipython-input-4-0a9a5fc0dc1c>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  random_forest.fit(X_train, y_train)


(87.8, 69.58)

### 로지스틱 회귀분석

In [5]:
log_reg=LogisticRegression()
log_reg.fit(X_train, y_train)
Y_pred=log_reg.predict(X_test)
train_acc_log=round(log_reg.score(X_train, y_train)*100, 2)
test_acc_log=round(log_reg.score(X_test, y_test)*100, 2)
train_acc_log, test_acc_log

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(80.3, 79.61)

### 나이브 베이지안

In [6]:
gaussian=GaussianNB()
gaussian.fit(X_train, y_train)
Y_pred=gaussian.predict(X_test)
train_acc_gaussian=round(gaussian.score(X_train, y_train)*100, 2)
test_acc_gaussian=round(gaussian.score(X_test, y_test)*100, 2)
train_acc_gaussian,test_acc_gaussian

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(77.8, 75.4)

###K-Nearest Neighbor

In [7]:
knn=KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
Y_pred=knn.predict(X_test)
train_acc_knn=round(knn.score(X_train, y_train)*100, 2)
test_acc_knn=round(knn.score(X_test, y_test)*100, 2)
train_acc_knn,test_acc_knn

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


(84.0, 68.61)

### SVM 모델

In [8]:
svc=LinearSVC()
svc.fit(X_train, y_train)
Y_pred=svc.predict(X_test)
train_acc_svc=round(svc.score(X_train, y_train)*100, 2)
test_acc_svc=round(svc.score(X_test, y_test)*100, 2)
train_acc_svc,test_acc_svc

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


(80.3, 78.64)

### 퍼셉트론 모델

In [9]:
perceptron=Perceptron(max_iter=5)
perceptron.fit(X_train, y_train)
Y_pred=perceptron.predict(X_test)
train_acc_perceptron=round(perceptron.score(X_train, y_train)*100, 2)
test_acc_perceptron=round(perceptron.score(X_test, y_test)*100, 2)
train_acc_perceptron,test_acc_perceptron

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


(79.5, 81.23)

### 확률적 경사하강법 방법

In [10]:
sgd=linear_model.SGDClassifier(max_iter=5, tol=None)
sgd.fit(X_train, y_train)
Y_pred=sgd.predict(X_test)
sgd.score(X_train, y_train)
train_acc_sgd=round(sgd.score(X_train, y_train)*100, 2)
test_acc_sgd=round(sgd.score(X_test, y_test)*100, 2)
train_acc_sgd,test_acc_sgd

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(75.1, 80.58)

## 2. 모델 비교
###모델의 정확도 비교

In [13]:
results = pd.DataFrame({
 'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 'Random Forest','Naive Bayes', 'Perceptron', 'Stochastic Gradient Decent', 'Decision Tree'],
 'train_Score': [train_acc_svc, train_acc_knn, train_acc_log,train_acc_random_forest,
train_acc_gaussian, train_acc_perceptron, train_acc_sgd, train_acc_decision_tree],
 'test_Score': [test_acc_svc, test_acc_knn,test_acc_log, test_acc_random_forest, test_acc_gaussian, test_acc_perceptron, test_acc_sgd, test_acc_decision_tree]})
result_df=results.sort_values(by='train_Score', ascending=False)
result_df=result_df.set_index('Model')
result_df.head(10)

,train_Score,test_Score
Model,,
Random Forest,87.8,69.58
Decision Tree,87.8,69.58
KNN,84.0,68.61
Support Vector Machines,80.3,78.64
Logistic Regression,80.3,79.61
Perceptron,79.5,81.23
Naive Bayes,77.8,75.40
Stochastic Gradient Decent,75.1,80.58


###랜덤 포레스트 모델의 교차검증 실습

In [14]:
from sklearn.model_selection import cross_val_score
rf=RandomForestClassifier(n_estimators=100)
scores=cross_val_score(rf, X_train, y_train, cv=10, scoring="accuracy")
print("Scores:", scores)
print("Mean:", scores.mean())
print("Standard Deviation:", scores.std())

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed whe

Scores: [0.71 0.62 0.83 0.81 0.84 0.74 0.76 0.84 0.67 0.67]
Mean: 0.749
Standard Deviation: 0.07595393340703296


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
